# Computation of Granularity Adjustments of MDB Portfolios

In [1]:
import numpy as np
from scipy.optimize import fsolve, minimize
import matplotlib.pyplot as plt 
import pandas as pd
import copy
from tqdm import tqdm
%load_ext autoreload
%autoreload 2
%aimport IS_misc_functions
from IS_misc_functions import *
from misc_functions import *
from actuarial_training import *
from MtM_training import *
from scipy.optimize import minimize

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler,normalize
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# Create Portfolios

Read in everything

In [2]:
by_country = pd.read_excel("MDB portfolios.xlsx", sheet_name='by country')
maturity = pd.read_excel("MDB portfolios.xlsx", sheet_name='maturity')
summary = pd.read_excel("MDB portfolios.xlsx", sheet_name='summary of current info')
comments = pd.read_excel("MDB portfolios.xlsx", sheet_name='comments')
rating = pd.read_excel("MDB portfolios.xlsx", sheet_name='rating')
transition_matrix_SP = pd.read_csv("transition_matrix_SP.csv",delimiter = ",",index_col = 0)
transition_matrix_RC = pd.read_csv("transition_matrix_RC.csv",delimiter = ";",index_col = 0)

Preprocessing the Data

In [3]:
SP_dict = trans_matrix_to_dict(transition_matrix_SP)
transition_matrix_SP

,AAA,AA+,AA,AA-,A+,A,A-,BBB+,BBB,BBB-,BB+,BB,BB-,B+,B,B-,Cs,D
AAA,96.79,2.71,0.42,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.07,0.00,0.00,0.00,0.00,0.00,0.00
AA+,6.45,85.16,6.61,1.77,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
AA,0.00,6.22,85.17,6.74,0.52,0.42,0.10,0.52,0.00,0.00,0.00,0.31,0.00,0.00,0.00,0.00,0.00,0.00
AA-,0.00,0.00,7.82,83.45,7.16,0.17,0.50,0.17,0.00,0.17,0.44,0.00,0.00,0.11,0.00,0.00,0.00,0.00
A+,0.00,0.00,0.07,13.35,73.28,9.30,2.03,1.12,0.14,0.63,0.07,0.00,0.00,0.00,0.00,0.00,0.00,0.01
A,0.00,0.00,0.00,1.15,12.33,77.29,5.71,1.68,0.77,0.96,0.10,0.00,0.00,0.00,0.00,0.00,0.00,0.01
A-,0.00,0.00,0.00,0.00,0.94,11.47,77.82,6.94,0.41,1.57,0.67,0.16,0.00,0.00,0.00,0.00,0.00,0.02
BBB+,0.00,0.00,0.00,0.00,0.00,2.16,12.39,70.86,11.24,2.41,0.60,0.24,0.06,0.00,0.00,0.00,0.00,0.04
BBB,0.00,0.00,0.00,0.00,0.00,0.00,1.87,16.60,68.05,11.16,0.99,0.11,0.00,0.50,0.22,0.11,0.33,0.06
BBB-,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.93,14.94,74.69,6.50,2.13,0.27,0.08,0.15,0.12,0.08,0.11


In [4]:
RC_dict = trans_matrix_to_dict(transition_matrix_RC)
transition_matrix_RC

,AAA,AA+,AA,AA-,A+,A,A-,BBB+,BBB,BBB-,BB+,BB,BB-,B+,B,B-,Cs,D
AAA,96.79,2.71,0.42,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.07,0.00,0.00,0.00,0.00,0.00,0.00
AA+,6.45,85.16,6.61,1.77,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
AA,0.00,6.22,85.17,6.74,0.52,0.42,0.10,0.52,0.00,0.00,0.00,0.31,0.00,0.00,0.00,0.00,0.00,0.00
AA-,0.00,0.00,7.82,83.45,7.16,0.17,0.50,0.17,0.00,0.17,0.44,0.00,0.00,0.11,0.00,0.00,0.00,0.00
A+,0.00,0.00,0.07,13.35,73.28,9.30,2.03,1.12,0.14,0.63,0.07,0.00,0.00,0.00,0.00,0.00,0.00,0.00
A,0.00,0.00,0.00,1.15,12.33,77.30,5.71,1.68,0.77,0.96,0.10,0.00,0.00,0.00,0.00,0.00,0.00,0.00
A-,0.00,0.00,0.00,0.00,0.94,11.47,77.83,6.94,0.41,1.57,0.67,0.16,0.00,0.00,0.00,0.00,0.00,0.01
BBB+,0.00,0.00,0.00,0.00,0.00,2.16,12.40,70.87,11.25,2.41,0.60,0.24,0.06,0.00,0.00,0.00,0.00,0.01
BBB,0.00,0.00,0.00,0.00,0.00,0.00,1.87,16.61,68.08,11.17,0.99,0.11,0.00,0.50,0.22,0.11,0.33,0.02
BBB-,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.93,14.95,74.75,6.51,2.13,0.27,0.08,0.15,0.12,0.08,0.03


In [5]:
#Small Tolerance to avoid 0s
tolerance = 10e-10

# list of banks
bank_names = list(summary.iloc[:,0])

# Exclude EIB and NIB from being considered
bank_names = [b for b in bank_names if b != "EIB" and b!= "NIB"]


# SP Rating of Countries
country_sp = {rating["Country"][i].lower().replace(" ", ""): 
              str(rating["sp_cb"][i]).replace(" ", "")  for i in range(len(rating["sp_cb"]))}

def create_PD_dict(transition_matrix,unrated_b_minus = True):
    country_dict = country_sp #copy.copy(country_sp)

    # The PDs of Rating Classes
    Rating_PD = {transition_matrix.index[i]: np.maximum(transition_matrix["D"][i],tolerance)  for i in range(len(transition_matrix.index))}

    # Companies with D rating are not in default wrt other MDBs:
    Rating_PD["D"] = Rating_PD["Cs"]
    Rating_PD["SD"] = Rating_PD["Cs"]
    
    if unrated_b_minus:
        for country_rating in set(country_dict.values()):
            if country_rating in ["C","CC","CCC-","CCC","CCC+"]:
                Rating_PD[country_rating] = Rating_PD["Cs"]
            if country_rating in ['0',0]:
                Rating_PD[country_rating] = Rating_PD["B-"] 

#         for country_rating in set(country_dict.values()):
#             if country_rating not in Rating_PD.keys():
#                 Rating_PD[country_rating] = "nan"

        for country in country_dict:
            if country_dict[country] in ["C","CC","CCC-","CCC","CCC+"]:
                country_dict[country] = "Cs"
            if country_dict[country] in ['0',0]:
                country_dict[country] = "B-"
    
    elif unrated_b_minus == False:

        # Adjust the PDs further 
        for country_rating in set(country_dict.values()):
            if country_rating in ["C","CC",'0',0,"CCC-","CCC","CCC+"]:
                Rating_PD[country_rating] = Rating_PD["Cs"] 

        for country_rating in set(country_dict.values()):
            if country_rating not in Rating_PD.keys():
                Rating_PD[country_rating] = "nan"

        for country in country_dict:
            if country_dict[country] in ["C","CC",'0',0,"CCC-","CCC","CCC+"]:
                country_dict[country] = "Cs"

    # Assign the ratings to the countries
    countries_PD = {country.lower().replace(" ", ""): Rating_PD[country_dict[country]] for country in country_dict.keys()}
    return countries_PD

In [6]:
countries_PD_SP = create_PD_dict(transition_matrix_SP)
countries_PD_RC = create_PD_dict(transition_matrix_RC)

Diplay the used PDs

In [7]:
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(pd.DataFrame([countries_PD_SP,countries_PD_RC],index =["S&P PDs","RC PDs"]).transpose())

,S&P PDs,RC PDs
afghanistan,51.47,14.70
albania,1.46,0.42
algeria,0.90,0.26
andorra,0.04,0.01
angola,7.59,2.17
anguilla,7.59,2.17
antiguaandbarbuda,51.47,14.70
argentina,51.47,14.70
armenia,0.90,0.26
aruba,0.06,0.02


Find the spaces between banks

In [8]:
spaces = np.where(pd.isnull(by_country.iloc[:,3]))[0]
spaces[0] = 1
spaces = np.append(spaces,len(by_country.iloc[:,3]))

### Create the portfolios

Define a function to create the portfolios

In [9]:
def create_portfolio(q = 0.99, ELGD_val = 0.45,
                     transition_matrix = transition_matrix_SP,
                     countries_PD = countries_PD_SP,
                    constant_rho = False,
                    rho = 0.35):
    portfolios = {}
    for i in range(len(bank_names)):
        considered_countries = np.array(by_country.iloc[:,1][(spaces[i]+1):(spaces[i+1]-1)])
        PD = []
        EAD = []
        g = []
        for j in range(len(considered_countries)):
            country = considered_countries[j].lower().replace(" ", "")
            if country in countries_PD.keys() and countries_PD[country] != 'nan': # nur anlegen wenn existent
                PD.append(countries_PD[country] /100)
                EAD.append(by_country.iloc[:,3][(spaces[i]+1)+j])
                g.append(S-list(transition_matrix.index).index(country_sp[country]))
        EAD = EAD/np.sum(np.array(EAD,dtype = np.float64))
        PD = np.array(PD)
        ELGD = np.array([ELGD_val]*len(EAD))
        if constant_rho:
            def to_solve(om):
                b = (0.11852-0.05478*np.log(PD))**2
                R_corr = rho
                K = ((ELGD)*norm.cdf(np.sqrt(1/(1-R_corr))*norm.ppf(PD)+np.sqrt(R_corr/(1-R_corr))*norm.ppf(q))-PD*ELGD)
                alpha_X =  gamma.ppf(q,a =0.25, scale = 1/0.25)
                return  (np.abs(ELGD*PD*om*(alpha_X-1)-K))
        else:
            def to_solve(om):
                b = (0.11852-0.05478*np.log(PD))**2
                R_corr = 0.12*(1-np.exp(-50*PD))/(1-np.exp(-50))+0.24*(1-(1-np.exp(-50*PD))/(1-np.exp(-50)))
                K = ((ELGD)*norm.cdf(np.sqrt(1/(1-R_corr))*norm.ppf(PD)+np.sqrt(R_corr/(1-R_corr))*norm.ppf(q))-PD*ELGD)
                alpha_X =  gamma.ppf(q,a =0.25, scale = 1/0.25)
                return  (np.abs(ELGD*PD*om*(alpha_X-1)-K))
        omega = fsolve(to_solve,[0.25]*len(EAD))
        omega = np.minimum(omega,1)
        #bnds = [(0,1) for i in range(len(EAD))]
        #omega = minimize(to_solve,[1]*len(EAD), bounds=bnds).x
        portfolios[bank_names[i]] = [EAD,ELGD,PD,omega,g]
    return portfolios

Create the portfolios

In [10]:
portfolios_099 = create_portfolio(q = 0.99,transition_matrix = transition_matrix_SP,countries_PD = countries_PD_SP)
portfolios_0999 = create_portfolio(q = 0.999,transition_matrix = transition_matrix_SP,countries_PD = countries_PD_SP)
portfolios_099_RC = create_portfolio(q = 0.99,transition_matrix = transition_matrix_RC,countries_PD = countries_PD_RC)
portfolios_0999_RC = create_portfolio(q = 0.999,transition_matrix = transition_matrix_RC,countries_PD = countries_PD_RC)

# Create Portfolios for rho = 0.12
portfolios_099_rho012 = create_portfolio(q = 0.99,transition_matrix = transition_matrix_SP,countries_PD = countries_PD_SP,
                                        constant_rho = True,rho = 0.12)
portfolios_0999_rho012 = create_portfolio(q = 0.999,transition_matrix = transition_matrix_SP,countries_PD = countries_PD_SP,
                                        constant_rho = True,rho = 0.12)
portfolios_099_RC_rho012 = create_portfolio(q = 0.99,transition_matrix = transition_matrix_RC,countries_PD = countries_PD_RC,
                                        constant_rho = True,rho = 0.12)
portfolios_0999_RC_rho012 = create_portfolio(q = 0.999,transition_matrix = transition_matrix_RC,countries_PD = countries_PD_RC,
                                        constant_rho = True,rho = 0.12)

# Create Portfolios for rho = 0.24
portfolios_099_rho024 = create_portfolio(q = 0.99,transition_matrix = transition_matrix_SP,countries_PD = countries_PD_SP,
                                        constant_rho = True,rho = 0.24)
portfolios_0999_rho024 = create_portfolio(q = 0.999,transition_matrix = transition_matrix_SP,countries_PD = countries_PD_SP,
                                        constant_rho = True,rho = 0.24)
portfolios_099_RC_rho024 = create_portfolio(q = 0.99,transition_matrix = transition_matrix_RC,countries_PD = countries_PD_RC,
                                        constant_rho = True,rho = 0.24)
portfolios_0999_RC_rho024 = create_portfolio(q = 0.999,transition_matrix = transition_matrix_RC,countries_PD = countries_PD_RC,
                                        constant_rho = True,rho = 0.24)

# Create Portfolios for rho = 0.35
portfolios_099_rho035 = create_portfolio(q = 0.99,transition_matrix = transition_matrix_SP,countries_PD = countries_PD_SP,
                                        constant_rho = True,rho = 0.35)
portfolios_0999_rho035 = create_portfolio(q = 0.999,transition_matrix = transition_matrix_SP,countries_PD = countries_PD_SP,
                                        constant_rho = True,rho = 0.35)
portfolios_099_RC_rho035 = create_portfolio(q = 0.99,transition_matrix = transition_matrix_RC,countries_PD = countries_PD_RC,
                                        constant_rho = True,rho = 0.35)
portfolios_0999_RC_rho035 = create_portfolio(q = 0.999,transition_matrix = transition_matrix_RC,countries_PD = countries_PD_RC,
                                        constant_rho = True,rho = 0.35)

C:\Users\jul_ses\Anaconda3\envs\tf_cpu\lib\site-packages\scipy\optimize\_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


# Test everything

First load the models

In [11]:
# Actuarial 
model_actuarial_099 = keras.models.load_model("model_actuarial_099.h5")
model_actuarial_0999 = keras.models.load_model("model_actuarial_0999.h5")
model_actuarial_099.compile()
model_actuarial_0999.compile()

#Mark to Market
model_mtm_099 = keras.models.load_model("model_mtm_099.h5")
model_mtm_0999 = keras.models.load_model("model_mtm_0999.h5")
model_mtm_099.compile()
model_mtm_0999.compile()

Define a Function for the computations

In [12]:
def compute_GA(q=0.99,ELGD_const = 0.45,
               portfolios = portfolios_099,
               model_mtm = model_mtm_099,
               model_actuarial = model_actuarial_099,
               trans_dict = SP_dict,
               compute_mtm=True,
               constant_rho = False,
               constant_rho_val = 0.35,
               relative_to_UL = False,
              n_mc = 100000, # For Actuarial Approach
              n_mc_mtm = 100000,
              n_mc_mtm_n1 = 50,
              maturity = 1,
              xi = 0.25):
    GA_approx = []
    GA_approx_simplified_LGD_const = []
    GA_approx_simplified = []
    GA_approx_VLGD = []
    GA_approx_2nd = []
    GA_approx_omegas = []
    GA_IRB_MC_list = []
    GA_IRB_MC_LGD_const_list = []
    GA_NN = []
    GA_MC = []
    GA_MC_LGD_const = []
    GA_MC_omegas = []
    GA_MC_LGD_const_omegas = []
    GA_MTM_DO = []
    GA_MTM_DO_LGD_const = []
    GA_MTM_approx_DO = []
    GA_MTM_approx = []
    GA_MTM_approx_DO_LGD_const = []
    GA_MTM_approx_LGD_const = []
    GA_MTM = []
    GA_MTM_LGD_const = []
    GA_MTM_NN = []

    for portfolio in portfolios:
        EAD,ELGD,PD,omega,g = portfolios[portfolio]
        ELGD = np.array([ELGD_const]*len(EAD))
        if len(EAD)>100: # Choose 100 indices
            indices = np.random.choice(range(len(EAD)-1), 100,replace = False)
            EAD = EAD[indices]
            PD = PD[indices]
            omega = omega[indices]
            ELGD = ELGD[indices]
            g = np.array(g)[indices]
        EAD = EAD/np.sum(EAD)
        if constant_rho == False:
            if relative_to_UL:
                # Determine Kstar
                def to_solve(R_corr):
                    K = ((ELGD)*norm.cdf(np.sqrt(1/(1-R_corr))*norm.ppf(PD)+np.sqrt(R_corr/(1-R_corr))*norm.ppf(q))-PD*ELGD)
                    alpha_X =  gamma.ppf(q,a =0.25, scale = 1/0.25)
                    return ELGD*PD*omega*(alpha_X-1)-K
                R_corr = fsolve(to_solve,[0.18]*len(EAD))
                K = ((ELGD)*norm.cdf(np.sqrt(1/(1-R_corr))*norm.ppf(PD)+np.sqrt(R_corr/(1-R_corr))*norm.ppf(q))-PD*ELGD)
                K_star_omega = np.sum(K*EAD)
                R_corr = 0.12*(1-np.exp(-50*PD))/(1-np.exp(-50))+0.24*(1-(1-np.exp(-50*PD))/(1-np.exp(-50)))
                K = ((ELGD)*norm.cdf(np.sqrt(1/(1-R_corr))*norm.ppf(PD)+np.sqrt(R_corr/(1-R_corr))*norm.ppf(q))-PD*ELGD)
                K_star_IRB = np.sum(K*EAD)
            if compute_mtm:
                GA_MTM.append(MC_IS_MtM(ELGD,EAD,
                                            rho = 0.12*(1-np.exp(-50*PD))/(1-np.exp(-50))+0.24*(1-(1-np.exp(-50*PD))/(1-np.exp(-50))),
                                            c = np.array([0.01]*len(EAD)),
                                            g = np.array(g),
                                            D= np.array([maturity]*len(EAD)),                                        
                                            trans_dict = trans_dict,
                                            r= r_nelson,
                                            q=q,T=1,n = n_mc_mtm,n1  = n_mc_mtm_n1,default_only=False))
                GA_MTM_LGD_const.append(MC_IS_MtM(ELGD,EAD,
                            rho = 0.12*(1-np.exp(-50*PD))/(1-np.exp(-50))+0.24*(1-(1-np.exp(-50*PD))/(1-np.exp(-50))),
                            c = np.array([0.01]*len(EAD)),
                            g = np.array(g),
                            D= np.array([maturity]*len(EAD)),                                        
                            trans_dict = trans_dict,
                            r= r_nelson,
                            q=q,T=1,n = n_mc_mtm,n1  = n_mc_mtm_n1,default_only=False,LGD_constant = True))
                GA_MTM_approx_LGD_const.append(GA_GM(PD = PD, ELGD =ELGD, A = EAD, M= np.array([maturity]*len(EAD)),
                      q=q,        
                     r = r_nelson,
                     T=1,
                     g= np.array(g),
                     trans_prob=trans_dict["trans_prob"],#N times S matrix
                     psi = 0.4, # Market Sharpe Ratio,
                     rho = 0.12*(1-np.exp(-50*PD))/(1-np.exp(-50))+0.24*(1-(1-np.exp(-50*PD))/(1-np.exp(-50))),
                     c = 0.01,
                    nu = 0.0,
                     S = S, # Number of States of Rating
                     default_only = False,
                      LGD_constant = True))
                GA_MTM_approx.append(GA_GM(PD = PD, ELGD =ELGD, A = EAD, M= np.array([maturity]*len(EAD)),
                      q=q,        
                     r = r_nelson,
                     T=1,
                     g= np.array(g),
                     trans_prob=trans_dict["trans_prob"],#N times S matrix
                     psi = 0.4, # Market Sharpe Ratio,
                     rho = 0.12*(1-np.exp(-50*PD))/(1-np.exp(-50))+0.24*(1-(1-np.exp(-50*PD))/(1-np.exp(-50))),
                     c = 0.01,
                    nu = 0.25,
                     S = S, # Number of States of Rating
                     default_only = False,
                      LGD_constant = False))
                GA_MTM_DO.append(MC_IS_MtM(ELGD,EAD,
                                            rho = 0.12*(1-np.exp(-50*PD))/(1-np.exp(-50))+0.24*(1-(1-np.exp(-50*PD))/(1-np.exp(-50))),
                                            c = np.array([0.01]*len(EAD)),
                                            g = np.array(g),
                                            D= np.array([maturity]*len(EAD)),
                                            trans_dict = trans_dict,
                                            r= r_nelson,
                                            q=q,T=1,n = n_mc_mtm,n1 = n_mc_mtm_n1,default_only=True))
                GA_MTM_DO_LGD_const.append(MC_IS_MtM(ELGD,EAD,
                                            rho = 0.12*(1-np.exp(-50*PD))/(1-np.exp(-50))+0.24*(1-(1-np.exp(-50*PD))/(1-np.exp(-50))),
                                            c = np.array([0.01]*len(EAD)),
                                            g = np.array(g),
                                            D= np.array([maturity]*len(EAD)),
                                            trans_dict = trans_dict,
                                            r= r_nelson,
                                            q=q,T=1,n = n_mc_mtm,n1 = n_mc_mtm_n1,default_only=True,LGD_constant = True))
                GA_MTM_approx_DO_LGD_const.append(GA_GM(PD = PD, ELGD =ELGD, A = EAD, M= np.array([maturity]*len(EAD)),
                      q=q,        
                     r = r_nelson,
                     T=1,
                     g= np.array(g),
                     trans_prob=trans_dict["trans_prob"],#N times S matrix
                     psi = 0.4, # Market Sharpe Ratio,
                     rho = 0.12*(1-np.exp(-50*PD))/(1-np.exp(-50))+0.24*(1-(1-np.exp(-50*PD))/(1-np.exp(-50))),
                     c = 0.01,
                    nu =0,               
                     S = S, # Number of States of Rating
                     default_only = True,
                      LGD_constant = True))
                GA_MTM_approx_DO.append(GA_GM(PD = PD, ELGD =ELGD, A = EAD, M= np.array([maturity]*len(EAD)),
                      q=q,        
                     r = r_nelson,
                     T=1,
                     g= np.array(g),
                     trans_prob=trans_dict["trans_prob"],#N times S matrix
                     psi = 0.4, # Market Sharpe Ratio,
                     rho = 0.12*(1-np.exp(-50*PD))/(1-np.exp(-50))+0.24*(1-(1-np.exp(-50*PD))/(1-np.exp(-50))),
                     c = 0.01,
                    nu =0.25,               
                     S = S, # Number of States of Rating
                     default_only = True,
                      LGD_constant = False))
            
            GA_IRB_MC_list.append(GA_IRB_MC(PD, # Vector of default probabillities
                  ELGD, #Vector of expected losses given default,
                  EAD, # Vector of Exposures at default,
                  q = q, # quantile level
                  N_sim = n_mc,
                  nu = 0.25,
                  LGD_constant = False,
                  M = maturity))
            
            GA_IRB_MC_LGD_const_list.append(GA_IRB_MC(PD, # Vector of default probabillities
                  ELGD, #Vector of expected losses given default,
                  EAD, # Vector of Exposures at default,
                  q = q, # quantile level
                  N_sim = n_mc,
                  LGD_constant = True,
                  M = maturity))
            
            GA_approx_simplified.append(GA_GL(PD,
                                           ELGD,
                                           EAD, maturity, # Vector of Maturities
                  q = q, # quantile level
                  xi = xi, # precision parameter,
                  nu = 0.25,# recovery parameter
                  rho = omega,
                  simplified = True,
                  LGD_constant = False,
                  IRB = True))
            GA_approx_simplified_LGD_const.append(GA_GL(PD,
                                           ELGD,
                                           EAD, maturity, # Vector of Maturities
                  q = q, # quantile level
                  xi = xi, # precision parameter,
                  nu = 0.25,# recovery parameter
                  rho = omega,
                  simplified = True,
                  LGD_constant = True,
                  IRB = True))
            
            GA_approx_VLGD.append(GA_GL(PD,
                                           ELGD,
                                           EAD, maturity, # Vector of Maturities
                  q = q, # quantile level
                  xi = xi, # precision parameter,
                  nu = 0.25,# recovery parameter
                  rho = omega,
                  LGD_constant = False,
                  IRB = True))
            GA_approx_2nd.append(GA_GL(PD,
                                           ELGD,
                                           EAD, maturity, # Vector of Maturities
                  q = q, # quantile level
                  xi = xi, # precision parameter,
                  nu = 0.25,# recovery parameter
                  rho = omega,
                  LGD_constant = False,
                  second_order = True,
                  IRB = True))

            GA_approx.append(GA_GL(PD,
                                           ELGD,
                                           EAD, maturity, # Vector of Maturities
                  q = q, # quantile level
                  xi = xi, # precision parameter,
                  nu = 0.25,# recovery parameter
                  rho = omega,
                  LGD_constant = True,
                  IRB = True)) 
            GA_approx_omegas.append(GA_GL(PD,
                                           ELGD,
                                           EAD, maturity, # Vector of Maturities
                  q = q, # quantile level
                  xi = xi, # precision parameter,
                  nu = 0.25,# recovery parameter
                  rho = omega,
                  LGD_constant = True,
                  IRB = False))
            if relative_to_UL:
                GA_IRB_MC_list[-1] = GA_IRB_MC_list[-1]/(K_star_IRB+GA_IRB_MC_list[-1])                
                GA_IRB_MC_LGD_const_list[-1] = GA_IRB_MC_LGD_const_list[-1]/(K_star_IRB+GA_IRB_MC_LGD_const_list[-1])
                GA_approx_simplified[-1] = GA_approx_simplified[-1]/(K_star_IRB+GA_approx_simplified[-1])
                GA_approx_simplified_LGD_const[-1]   = GA_approx_simplified_LGD_const[-1]/(K_star_IRB+GA_approx_simplified_LGD_const[-1])
                GA_approx_VLGD[-1] = GA_approx_VLGD[-1]/(K_star_IRB+GA_approx_VLGD[-1])
                GA_approx[-1] = GA_approx[-1]/(K_star_IRB+GA_approx[-1])
                GA_approx_omegas[-1] = GA_approx_omegas[-1]/(K_star_omega+GA_approx_omegas[-1])
            # Adjust Input for NNs:
            rho = 0.12*(1-np.exp(-50*PD))/(1-np.exp(-50))+0.24*(1-(1-np.exp(-50*PD))/(1-np.exp(-50)))
            rho = np.reshape(np.append(rho,[0]*(100-len(EAD))),(1,100))
            c = np.array([0.01]*len(EAD))
            c = np.reshape(np.append(c,[0]*(100-len(EAD))),(1,100))
            g = np.array(g)
            g = np.reshape(np.append(g,[0]*(100-len(EAD))),(1,100))
            D= np.array([1]*len(EAD))
            D=  np.reshape(np.append(D,[0]*(100-len(EAD))),(1,100))
            EAD = np.reshape(np.append(EAD,[0]*(100-len(EAD))),(1,100))
            ELGD = np.reshape(np.append(ELGD,[0]*(100-len(ELGD))),(1,100))
            PD = np.reshape(np.append(PD,[0]*(100-len(PD))),(1,100))
            omega = np.reshape(np.append(omega,[0]*(100-len(omega))),(1,100))
            Input = np.concatenate([ELGD ,EAD ,PD ,omega ],axis =1)
            Input_mtm = np.concatenate([ELGD ,EAD,rho ,c, g,  D ],axis =1)
            GA_NN.append(model_actuarial.predict(Input)[0][0])
            GA_MC_LGD_const.append(MC_IS_actuarial(ELGD ,EAD ,PD ,omega,q=q,IRB = True,n = n_mc,LGD_constant = True)[0])
            GA_MC_LGD_const_omegas.append(MC_IS_actuarial(ELGD ,EAD ,PD ,omega,q=q,IRB = False,n = n_mc,LGD_constant = True)[0])
            GA_MC.append(MC_IS_actuarial(ELGD ,EAD ,PD ,omega,q=q,IRB = True,n = n_mc)[0])
            GA_MC_omegas.append(MC_IS_actuarial(ELGD ,EAD ,PD ,omega,q=q,IRB = False,n = n_mc)[0])
            if relative_to_UL:
                GA_NN[-1] = GA_NN[-1]/(K_star_omega+GA_NN[-1] )
                GA_MC[-1] =  GA_MC[-1]/(K_star_IRB+GA_MC[-1] )
                GA_MC_omegas[-1] =  GA_MC_omegas[-1]/(K_star_omega+GA_MC_omegas[-1])
                GA_MC_LGD_const[-1] =  GA_MC_LGD_const[-1]/(K_star_IRB+GA_MC_LGD_const[-1] )
                GA_MC_LGD_const_omegas[-1] =  GA_MC_LGD_const_omegas[-1]/(K_star_omega+GA_MC_LGD_const_omegas[-1])
            if compute_mtm:
                GA_MTM_NN.append(model_mtm.predict(Input_mtm)[0][0])
            if compute_mtm:
                df = pd.DataFrame([[len(portfolios[portfolio][0]) for portfolio in portfolios],
                                                    GA_approx,GA_approx_omegas,
                                      GA_approx_VLGD,GA_approx_2nd, GA_approx_simplified,GA_approx_simplified_LGD_const,
                                      GA_NN,
                                      GA_MC,GA_MC_omegas,GA_MC_LGD_const,GA_MC_LGD_const_omegas,
                                   GA_IRB_MC_list,GA_IRB_MC_LGD_const_list,
                                      GA_MTM_DO,GA_MTM_approx_DO,GA_MTM_DO_LGD_const, GA_MTM_approx_DO_LGD_const,
                                     GA_MTM,GA_MTM_approx,GA_MTM_LGD_const, GA_MTM_approx_LGD_const,GA_MTM_NN], 
                                     columns=bank_names, 
                                     index = ["Borrowers",
                                              "GA approx (rho from IRB, VLGD  = 0)",
                                              "GA approx (omega as Input , VLGD = 0)", 
                                              "GA approx (rho from IRB, nu = 0.25)",
                                              "GA approx (rho from IRB, nu = 0.25, 2nd order)",
                                              "GA approx (simplified, nu = 0.25)",
                                              "GA approx (simplified, nu = 0)",
                                              "GA CR+ NN", 
                                              "GA Monte Carlo (CR+, rho from IRB, nu = 0.25)",
                                             "GA Monte Carlo (CR+, omega as Input, nu = 0.25)",
                                             "GA Monte Carlo (CR+, rho from IRB, LGD constant)",
                                             "GA Monte Carlo (CR+, omega as Input, LGD constant)",
                                            "GA Monte Carlo (IRB, nu = 0.25)",
                                            "GA Monte Carlo (IRB, LGD constant)",
                                             "GA MTM Monte Carlo (Default Only, nu = 0.25)",
                                             "GA MTM approx. (Default Only, nu = 0.25)",
                                             "GA MTM Monte Carlo (Default Only, nu = 0)",
                                             "GA MTM Approx. (Default Only , nu = 0)",
                                            "GA MTM Monte Carlo, nu = 0.25",
                                            "GA MTM Approx. , nu = 0.25",
                                            "GA MTM Monte Carlo, nu = 0",
                                             "GA MTM Approx. , nu = 0",
                                              "GA MTM NN"])
            else:
                df = pd.DataFrame([[len(portfolios[portfolio][0]) for portfolio in portfolios],
                                                    GA_approx,GA_approx_omegas,
                                      GA_approx_VLGD,GA_approx_2nd,GA_approx_simplified,GA_approx_simplified_LGD_const,
                                      GA_NN,
                                      GA_MC,GA_MC_omegas,GA_MC_LGD_const,GA_MC_LGD_const_omegas,
                                   GA_IRB_MC_list,GA_IRB_MC_LGD_const_list], 
                                     columns=bank_names, 
                                     index = ["Borrowers",
                                              "GA approx (rho from IRB, VLGD  = 0)",
                                              "GA approx (omega as Input , VLGD = 0)", 
                                              "GA approx (rho from IRB, nu = 0.25)",
                                              "GA approx (rho from IRB, nu = 0.25, 2nd order)",
                                              "GA approx (simplified, nu = 0.25)",                                              
                                              "GA approx (simplified, nu = 0)",
                                              "GA CR+ NN", 
                                              "GA Monte Carlo (CR+, rho from IRB, nu = 0.25)",
                                             "GA Monte Carlo (CR+, omega as Input, nu = 0.25)",
                                             "GA Monte Carlo (CR+, rho from IRB, LGD constant)",
                                             "GA Monte Carlo (CR+, omega as Input, LGD constant)",
                                            "GA Monte Carlo (IRB model, nu = 0.25)",
                                            "GA Monte Carlo (IRB model,  LGD constant)"])

                
                
                
                
        elif constant_rho == True:
            if relative_to_UL:
                R_corr = np.array([constant_rho_val]*len(EAD))
                K = ((ELGD)*norm.cdf(np.sqrt(1/(1-R_corr))*norm.ppf(PD)+np.sqrt(R_corr/(1-R_corr))*norm.ppf(q))-PD*ELGD)
                K_star = np.sum(K*EAD)
            if compute_mtm:
                GA_MTM.append(MC_IS_MtM(ELGD,EAD,
                                            rho = np.array([constant_rho_val]*len(EAD)),
                                            c = np.array([0.01]*len(EAD)),
                                            g = np.array(g),
                                            D= np.array([maturity]*len(EAD)),                                        
                                            trans_dict = trans_dict,
                                            r= r_nelson,
                                            q=q,T=1,n = n_mc_mtm,n1 = n_mc_mtm_n1,default_only=False))
                GA_MTM_LGD_const.append(MC_IS_MtM(ELGD,EAD,
                                rho = np.array([constant_rho_val]*len(EAD)),
                                c = np.array([0.01]*len(EAD)),
                                g = np.array(g),
                                D= np.array([maturity]*len(EAD)),                                        
                                trans_dict = trans_dict,
                                r= r_nelson,
                                q=q,T=1,n = n_mc_mtm,n1 = n_mc_mtm_n1,default_only=False,LGD_constant = True)) 
                GA_MTM_approx_LGD_const.append(GA_GM(PD = PD, ELGD =ELGD, A = EAD, M= np.array([maturity]*len(EAD)),
                      q=q,        
                     r = r_nelson,
                     T=1,
                     g= np.array(g),
                     trans_prob=trans_dict["trans_prob"],#N times S matrix
                     psi = 0.4, # Market Sharpe Ratio,
                     rho = np.array([constant_rho_val]*len(EAD)),
                     c = 0.01,                                    
                     S = S, # Number of States of Rating
                     default_only = False,
                      LGD_constant = True))
                GA_MTM_approx.append(GA_GM(PD = PD, ELGD =ELGD, A = EAD, M= np.array([maturity]*len(EAD)),
                      q=q,        
                     r = r_nelson,
                     T=1,
                     g= np.array(g),
                     trans_prob=trans_dict["trans_prob"],#N times S matrix
                     psi = 0.4, # Market Sharpe Ratio,
                     rho = np.array([constant_rho_val]*len(EAD)),
                     nu =0.25,
                     c = 0.01,                                    
                     S = S, # Number of States of Rating
                     default_only = False,
                      LGD_constant = False))
                GA_MTM_DO.append(MC_IS_MtM(ELGD,EAD,
                                            rho = np.array([constant_rho_val]*len(EAD)),
                                            c = np.array([0.01]*len(EAD)),
                                            g = np.array(g),
                                            D= np.array([maturity]*len(EAD)),                                           
                                            trans_dict = trans_dict,
                                            r= r_nelson,
                                            q=q,T=1,n = n_mc_mtm,n1 = n_mc_mtm_n1,default_only=True))
                GA_MTM_DO_LGD_const.append(MC_IS_MtM(ELGD,EAD,
                                rho = np.array([constant_rho_val]*len(EAD)),
                                c = np.array([0.01]*len(EAD)),
                                g = np.array(g),
                                D= np.array([maturity]*len(EAD)),                                           
                                trans_dict = trans_dict,
                                r= r_nelson,
                                q=q,T=1,n = n_mc_mtm,n1 = n_mc_mtm_n1,default_only=True,LGD_constant = True))
                GA_MTM_approx_DO.append(GA_GM(PD = PD, ELGD =ELGD, A = EAD, M= np.array([maturity]*len(EAD)),
                      q=q,        
                     r = r_nelson,
                     T=1,
                     g= np.array(g),
                     trans_prob=trans_dict["trans_prob"],#N times S matrix
                     psi = 0.4, # Market Sharpe Ratio,
                     rho = np.array([constant_rho_val]*len(EAD)),
                     c = 0.01,
                     S = S, # Number of States of Rating
                     default_only = True,
                      LGD_constant = True))
                GA_MTM_approx_DO_LGD_const.append(GA_GM(PD = PD, ELGD =ELGD, A = EAD, M= np.array([maturity]*len(EAD)),
                      q=q,        
                     r = r_nelson,
                     T=1,
                     g= np.array(g),
                     trans_prob=trans_dict["trans_prob"],#N times S matrix
                     psi = 0.4, # Market Sharpe Ratio,
                     rho = np.array([constant_rho_val]*len(EAD)),
                     c = 0.01,
                     nu = 0.25,
                     S = S, # Number of States of Rating
                     default_only = True,
                      LGD_constant = False))
            GA_IRB_MC_list.append(GA_IRB_MC(PD, # Vector of default probabillities
                  ELGD, #Vector of expected losses given default,
                  EAD, # Vector of Exposures at default,
                  q = q, # quantile level
                  N_sim = n_mc,
                  constant_rho = True,
                  constant_rho_val = constant_rho_val,
                  M = maturity))
            GA_IRB_MC_LGD_const_list.append(GA_IRB_MC(PD, # Vector of default probabillities
                  ELGD, #Vector of expected losses given default,
                  EAD, # Vector of Exposures at default,
                  q = q, # quantile level
                  N_sim = n_mc,
                  constant_rho = True,
                  constant_rho_val = constant_rho_val,
                  nu = 0.25,
                  LGD_constant = True,
                  M = maturity))
            GA_approx_simplified.append(GA_GL(PD,
                                           ELGD,
                                           EAD, maturity, # Vector of Maturities
                  q = q, # quantile level
                  xi = xi, # precision parameter,
                  nu = 0.25,# recovery parameter
                  rho = omega,
                  simplified = True,
                  LGD_constant = False,
                  IRB = True,
                  constant_rho = True,
                  constant_rho_val = constant_rho_val))
            GA_approx_simplified_LGD_const.append(GA_GL(PD,
                                           ELGD,
                                           EAD, maturity, # Vector of Maturities
                  q = q, # quantile level
                  xi = xi, # precision parameter,
                  nu = 0.25,# recovery parameter
                  rho = omega,
                  simplified = True,
                  LGD_constant = True,
                  IRB = True,
                  constant_rho = True,
                  constant_rho_val = constant_rho_val))

            GA_approx.append(GA_GL(PD,
                                           ELGD,
                                           EAD, maturity, # Vector of Maturities
                  q = q, # quantile level
                  xi = xi, # precision parameter,
                  nu = 0.25,# recovery parameter
                  rho = omega,
                  LGD_constant = True,
                  IRB = True,
                  constant_rho = True,
                  constant_rho_val = constant_rho_val)) 
    
            GA_approx_omegas.append(GA_GL(PD,
                                           ELGD,
                                           EAD, maturity, # Vector of Maturities
                  q = q, # quantile level
                  xi = xi, # precision parameter,
                  nu = 0.25,# recovery parameter
                  rho = omega,
                  LGD_constant = True,
                  IRB = False,
                  constant_rho = False,
                  constant_rho_val = constant_rho_val)) 
            GA_approx_VLGD.append(GA_GL(PD,
                                           ELGD,
                                           EAD, maturity, # Vector of Maturities
                  q = q, # quantile level
                  xi = xi, # precision parameter,
                  nu = 0.25,# recovery parameter
                  rho = omega,
                  LGD_constant = False,
                  IRB = True,
                  constant_rho = True,
                  constant_rho_val = constant_rho_val))
            GA_approx_2nd.append(GA_GL(PD,
                                           ELGD,
                                           EAD, maturity, # Vector of Maturities
                  q = q, # quantile level
                  xi = xi, # precision parameter,
                  nu = 0.25,# recovery parameter
                  rho = omega,
                  LGD_constant = False,
                  second_order = True,
                  IRB = True,
                  constant_rho = True,
                  constant_rho_val = constant_rho_val))
            if relative_to_UL:
                GA_IRB_MC_list[-1] = GA_IRB_MC_list[-1]/(K_star+GA_IRB_MC_list[-1])
                GA_IRB_MC_LGD_const_list[-1] = GA_IRB_MC_LGD_const_list[-1]/(K_star+GA_IRB_MC_LGD_const_list[-1])
                GA_approx_simplified[-1] = GA_approx_simplified[-1]/(K_star+GA_approx_simplified[-1])
                GA_approx_simplified_LGD_const[-1] = GA_approx_simplified_LGD_const[-1]/(K_star+GA_approx_simplified_LGD_const[-1])
                GA_approx[-1] = GA_approx[-1]/(K_star+GA_approx[-1])
                GA_approx_omegas[-1] = GA_approx_omegas[-1]/(K_star+GA_approx_omegas[-1])
                GA_approx_VLGD[-1] =  GA_approx_VLGD[-1]/(K_star+GA_approx_VLGD[-1])
                GA_approx_2nd[-1] = GA_approx_2nd[-1]/(K_star+GA_approx_2nd[-1])
            # Adjust Input for NNs:
            rho = np.array([constant_rho_val]*len(EAD))
            rho = np.reshape(np.append(rho,[0]*(100-len(EAD))),(1,100))
            c = np.array([0.01]*len(EAD))
            c = np.reshape(np.append(c,[0]*(100-len(EAD))),(1,100))
            g = np.array(g)
            g = np.reshape(np.append(g,[0]*(100-len(EAD))),(1,100))
            D= np.array([1]*len(EAD))
            D=  np.reshape(np.append(D,[0]*(100-len(EAD))),(1,100))
            EAD = np.reshape(np.append(EAD,[0]*(100-len(EAD))),(1,100))
            ELGD = np.reshape(np.append(ELGD,[0]*(100-len(ELGD))),(1,100))
            PD = np.reshape(np.append(PD,[0]*(100-len(PD))),(1,100))
            omega = np.reshape(np.append(omega,[0]*(100-len(omega))),(1,100))
            Input = np.concatenate([ELGD ,EAD ,PD ,omega ],axis =1)
            Input_mtm = np.concatenate([ELGD ,EAD,rho ,c, g,  D ],axis =1)
            GA_NN.append(model_actuarial.predict(Input)[0][0])
            
            GA_MC_LGD_const.append(MC_IS_actuarial(ELGD ,EAD ,PD ,omega,q=q,constant_rho = True,
                  constant_rho_val = constant_rho_val,IRB = True,n = n_mc,LGD_constant = True)[0])
            GA_MC_LGD_const_omegas.append(MC_IS_actuarial(ELGD ,EAD ,PD ,omega,q=q, IRB = False,n = n_mc,LGD_constant = True)[0])
            GA_MC.append(MC_IS_actuarial(ELGD ,EAD ,PD ,omega,q=q,                  constant_rho = True,
                  constant_rho_val = constant_rho_val,IRB = True,n = n_mc)[0])
            GA_MC_omegas.append(MC_IS_actuarial(ELGD ,EAD ,PD ,omega,q=q,                  constant_rho = True,IRB = False,n = n_mc)[0])
            if relative_to_UL:
                GA_NN[-1] = GA_NN[-1]/(K_star+GA_NN[-1] )
                GA_MC[-1] =  GA_MC[-1]/(K_star+GA_MC[-1] )
                GA_MC_omegas[-1] =  GA_MC_omegas[-1]/(K_star+GA_MC_omegas[-1])
                GA_MC_LGD_const[-1] =  GA_MC_LGD_const[-1]/(K_star+GA_MC_LGD_const[-1] )
                GA_MC_LGD_const_omegas[-1] =  GA_MC_LGD_const_omegas[-1]/(K_star+GA_MC_LGD_const_omegas[-1])
            if compute_mtm:
                GA_MTM_NN.append(model_mtm.predict(Input_mtm)[0][0])
            if compute_mtm:
                df = pd.DataFrame([[len(portfolios[portfolio][0]) for portfolio in portfolios],
                                                    GA_approx,GA_approx_VLGD,GA_approx_2nd,GA_approx_omegas,
                                      GA_approx_simplified,GA_approx_simplified_LGD_const,
                                      GA_NN,
                                      GA_MC,GA_MC_omegas,GA_MC_LGD_const,GA_MC_LGD_const_omegas,
                                   GA_IRB_MC_list,GA_IRB_MC_LGD_const_list,
                                      GA_MTM_DO,GA_MTM_approx_DO,GA_MTM_DO_LGD_const, GA_MTM_approx_DO_LGD_const,
                                     GA_MTM,GA_MTM_approx,GA_MTM_LGD_const, GA_MTM_approx_LGD_const,GA_MTM_NN], 
                                     columns=bank_names, 
                                     index = ["Borrowers",
                                              "GA approx (rho constant, nu = 0)",
                                              "GA approx (rho constant, nu = 0.25)",
                                              "GA approx (rho constant, nu = 0.25 2nd order)",
                                              "GA approx (omega as Input, nu = 0)", 
                                              "GA approx (simplified, rho constant, nu = 0.25)",
                                              "GA approx (simplified, rho constant, nu = 0)",
                                              "GA CR+ NN", 
                                              "GA Monte Carlo (CR+, rho from IRB, nu = 0.25)",
                                             "GA Monte Carlo (CR+, omega as Input, nu = 0.25)",
                                             "GA Monte Carlo (CR+, rho from IRB, LGD constant)",
                                             "GA Monte Carlo (CR+, omega as Input, LGD constant)",
                                            "GA Monte Carlo (IRB model, rho constant, nu = 0.25)",
                                            "GA Monte Carlo (IRB model, rho constant, LGD constant)",
                                             "GA MTM Monte Carlo (Default Only, nu = 0.25)",
                                             "GA MTM approx. (Default Only, nu = 0.25)",
                                             "GA MTM Monte Carlo (Default Only, nu = 0)",
                                             "GA MTM Approx. (Default Only , nu = 0)",
                                            "GA MTM Monte Carlo, nu = 0.25",
                                            "GA MTM Approx. , nu = 0.25",
                                            "GA MTM Monte Carlo, nu = 0",
                                             "GA MTM Approx. , nu = 0",
                                              "GA MTM NN"])
            else:
                df = pd.DataFrame([[len(portfolios[portfolio][0]) for portfolio in portfolios],
                                                    GA_approx,GA_approx_VLGD,GA_approx_2nd, GA_approx_omegas,
                                      GA_approx_simplified,GA_approx_simplified_LGD_const,
                                      GA_NN,
                                      GA_MC,GA_MC_omegas,GA_MC_LGD_const,GA_MC_LGD_const_omegas,
                                   GA_IRB_MC_list,GA_IRB_MC_LGD_const_list], 
                                     columns=bank_names, 
                                     index = ["Borrowers",
                                              "GA approx (rho constant, nu = 0)",
                                              "GA approx (rho constant, nu = 0.25)",
                                              "GA approx (rho constant, nu = 0.25 2nd order)",
                                              "GA approx (omega as Input, nu = 0)", 
                                              "GA approx (simplified, rho constant, nu = 0.25)",
                                              "GA approx (simplified, rho constant, nu = 0)",
                                              "GA CR+ NN", 
                                              "GA Monte Carlo (CR+, rho from IRB, nu = 0.25)",
                                             "GA Monte Carlo (CR+, omega as Input, nu = 0.25)",
                                             "GA Monte Carlo (CR+, rho from IRB, LGD constant)",
                                             "GA Monte Carlo (CR+, omega as Input, LGD constant)",
                                            "GA Monte Carlo (IRB model, rho constant, nu = 0.25)",
                                            "GA Monte Carlo (IRB model, rho constant, LGD constant)"]) 
                
    return df

Define a Function to calibrate xi

In [13]:
def calibrate_xi(q=0.99,ELGD_const = 0.45,
               portfolios = portfolios_099,
               model_mtm = model_mtm_099,
               model_actuarial = model_actuarial_099,
               trans_dict = SP_dict,
              n_mc = 100000, # For Actuarial Approach
              n_mc_mtm = 100000,
              n_mc_mtm_n1 = 50,
              maturities = 1):
    GA_approx_VLGD = []
    GA_MTM = []
    
    EAD_list = [portfolios[portfolio][0]/np.sum(portfolios[portfolio][0]) for portfolio in portfolios]
    ELGD_list = [portfolios[portfolio][1] for portfolio in portfolios]
    PD_list = [portfolios[portfolio][2] for portfolio in portfolios]
    omega_list = [portfolios[portfolio][3] for portfolio in portfolios]
    g_list = [portfolios[portfolio][4] for portfolio in portfolios]
    i = 0
    for portfolio in portfolios:
        EAD,ELGD,PD,omega,g = portfolios[portfolio]
        EAD = EAD/np.sum(EAD)
        maturity = maturities[i]       
        GA_MTM.append(MC_IS_MtM(ELGD,EAD,
                                    rho = 0.12*(1-np.exp(-50*PD))/(1-np.exp(-50))+0.24*(1-(1-np.exp(-50*PD))/(1-np.exp(-50))),
                                    c = np.array([0.01]*len(EAD)),
                                    g = np.array(g),
                                    D = np.array([maturity]*len(EAD)),                                        
                                    trans_dict = trans_dict,
                                    r= r_nelson,
                                    q=q,T=1,n = n_mc_mtm,n1  = n_mc_mtm_n1,default_only=False))
        i = i+1
    def to_solve(xi):
        return np.sum([np.abs(GA_GL(PD_list[i],ELGD_list[i],EAD_list[i], maturities[i],q = q,xi = xi,nu = 0.25,
          rho = omega_list[i],LGD_constant = False,IRB = True)-GA_MTM[i])**2 for i in range(len(GA_MTM))])
    optimal_xi =   minimize(to_solve,0.25,bounds  = [(0.0001,10)])       
    return optimal_xi.x[0]

# GA Computations for S&P Transition Matrix

In [14]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)

## 1.) q = 99%, ELGD = 0.45

In [15]:
df_99 = compute_GA(q=0.99,ELGD_const = 0.45,
               portfolios = portfolios_099,
               model_mtm = model_mtm_099,
               model_actuarial = model_actuarial_099)
display(df_99)

1/1 [==============================] - 0s 27ms/step


,CAF,ADB,AFDB,IDB,CDB,CABEI,EADB,IBRD,TDB,BOAD,EBRD
Borrowers,16.0000,38.0000,29.0000,26.0000,16.0000,11.0000,4.0000,78.0000,21.0000,8.0000,37.0000
"GA approx (rho from IRB, VLGD = 0)",0.1305,0.0895,0.0708,0.1147,0.0969,0.2768,0.2097,0.0297,0.1496,0.1436,0.0677
"GA approx (omega as Input , VLGD = 0)",0.1311,0.0902,0.0708,0.1153,0.0971,0.2771,0.2097,0.0298,0.1497,0.1436,0.0682
"GA approx (rho from IRB, nu = 0.25)",0.1923,0.1340,0.1042,0.1711,0.1385,0.4132,0.2786,0.0432,0.2260,0.2134,0.0985
"GA approx (rho from IRB, nu = 0.25, 2nd order)",0.4999,0.4368,0.2248,0.5667,0.2876,1.8063,1.5021,0.0787,0.4635,0.4698,0.2583
"GA approx (simplified, nu = 0.25)",0.1704,0.1169,0.0924,0.1498,0.1265,0.3614,0.2738,0.0387,0.1953,0.1874,0.0884
"GA approx (simplified, nu = 0)",0.1305,0.0895,0.0708,0.1147,0.0969,0.2768,0.2097,0.0297,0.1496,0.1436,0.0677
GA CR+ NN,0.0314,0.0178,0.0235,0.0266,0.0364,0.0469,0.0406,0.0094,0.0308,0.0609,0.0164
"GA Monte Carlo (CR+, rho from IRB, nu = 0.25)",0.0747,0.0437,0.0391,0.0549,0.0691,0.1239,0.1356,0.0191,0.0763,0.0791,0.0392
"GA Monte Carlo (CR+, omega as Input, nu = 0.25)",0.0721,0.0436,0.0390,0.0543,0.0696,0.1232,0.1366,0.0190,0.0819,0.0758,0.0383


Relative to UL

In [16]:
df_99_UL = compute_GA(q=0.99,ELGD_const = 0.45,
               portfolios = portfolios_099,
               model_mtm = model_mtm_099,
               model_actuarial = model_actuarial_099,
               compute_mtm= False,
               relative_to_UL = True)
display(df_99_UL)

1/1 [==============================] - 0s 20ms/step


,CAF,ADB,AFDB,IDB,CDB,CABEI,EADB,IBRD,TDB,BOAD,EBRD
Borrowers,16.0000,38.0000,29.0000,26.0000,16.0000,11.0000,4.0000,78.0000,21.0000,8.0000,37.0000
"GA approx (rho from IRB, VLGD = 0)",0.6946,0.7279,0.5530,0.7192,0.5786,0.8241,0.8242,0.4869,0.6033,0.6557,0.6382
"GA approx (omega as Input , VLGD = 0)",0.6969,0.7324,0.5544,0.7219,0.5796,0.8244,0.8242,0.4923,0.6036,0.6557,0.6424
"GA approx (rho from IRB, nu = 0.25)",0.7702,0.8002,0.6456,0.7925,0.6625,0.8749,0.8617,0.5800,0.6968,0.7390,0.7197
"GA approx (rho from IRB, nu = 0.25, 2nd order)",0.4999,0.4368,0.2248,0.5667,0.2876,1.8063,1.5021,0.0787,0.4635,0.4698,0.2583
"GA approx (simplified, nu = 0.25)",0.7480,0.7774,0.6176,0.7698,0.6419,0.8595,0.8596,0.5534,0.6651,0.7132,0.6972
"GA approx (simplified, nu = 0)",0.6946,0.7279,0.5530,0.7192,0.5786,0.8241,0.8242,0.4869,0.6033,0.6557,0.6382
GA CR+ NN,0.3549,0.3512,0.2924,0.3748,0.3406,0.4429,0.4757,0.2350,0.2388,0.4470,0.3014
"GA Monte Carlo (CR+, rho from IRB, nu = 0.25)",0.5573,0.5571,0.4046,0.5448,0.4961,0.6745,0.7519,0.3741,0.4378,0.5028,0.5050
"GA Monte Carlo (CR+, omega as Input, nu = 0.25)",0.5562,0.5665,0.4167,0.5468,0.4959,0.6747,0.7552,0.3859,0.4411,0.5117,0.5134


### 1.1) Constant Rho, Rho = 0.35

In [17]:
df_99_rho = compute_GA(q=0.99,ELGD_const = 0.45,
               portfolios = portfolios_099_rho035,
               model_mtm = model_mtm_099,
               model_actuarial = model_actuarial_099,
               constant_rho = True,
               constant_rho_val = 0.35)
display(df_99_rho)

1/1 [==============================] - 0s 23ms/step


,CAF,ADB,AFDB,IDB,CDB,CABEI,EADB,IBRD,TDB,BOAD,EBRD
Borrowers,16.0000,38.0000,29.0000,26.0000,16.0000,11.0000,4.0000,78.0000,21.0000,8.0000,37.0000
"GA approx (rho constant, nu = 0)",0.0956,0.0659,0.0517,0.0825,0.0729,0.2057,0.1818,0.0228,0.1111,0.1101,0.0516
"GA approx (rho constant, nu = 0.25)",0.1420,0.0998,0.0765,0.1242,0.1051,0.3096,0.2442,0.0333,0.1702,0.1647,0.0752
"GA approx (rho constant, nu = 0.25 2nd order)",0.2583,0.2330,0.1225,0.2815,0.1574,0.9039,0.7201,0.0484,0.2784,0.2810,0.1390
"GA approx (omega as Input, nu = 0)",0.0974,0.0677,0.0522,0.0850,0.0737,0.2120,0.1824,0.0230,0.1116,0.1115,0.0532
"GA approx (simplified, rho constant, nu = 0.25)",0.1248,0.0861,0.0675,0.1077,0.0952,0.2686,0.2374,0.0298,0.1451,0.1437,0.0673
"GA approx (simplified, rho constant, nu = 0)",0.0956,0.0659,0.0517,0.0825,0.0729,0.2057,0.1818,0.0228,0.1111,0.1101,0.0516
GA CR+ NN,0.0281,0.0176,0.0206,0.0241,0.0299,0.0408,0.0332,0.0097,0.0235,0.0528,0.0163
"GA Monte Carlo (CR+, rho from IRB, nu = 0.25)",0.0708,0.0436,0.0405,0.0550,0.0735,0.1234,0.1331,0.0195,0.0761,0.0797,0.0389
"GA Monte Carlo (CR+, omega as Input, nu = 0.25)",0.0705,0.0428,0.0416,0.0540,0.0704,0.1233,0.1346,0.0195,0.0768,0.0778,0.0386


Relative to UL

In [18]:
df_99_rho_UL = compute_GA(q=0.99,ELGD_const = 0.45,
               portfolios = portfolios_099_rho035,
               model_mtm = model_mtm_099,
               model_actuarial = model_actuarial_099,
               constant_rho = True,
               constant_rho_val = 0.35,
               compute_mtm= False,
               relative_to_UL = True)
display(df_99_rho_UL)

1/1 [==============================] - 0s 21ms/step


,CAF,ADB,AFDB,IDB,CDB,CABEI,EADB,IBRD,TDB,BOAD,EBRD
Borrowers,16.0000,38.0000,29.0000,26.0000,16.0000,11.0000,4.0000,78.0000,21.0000,8.0000,37.0000
"GA approx (rho constant, nu = 0)",0.4866,0.5538,0.3425,0.5238,0.3485,0.6826,0.6684,0.3033,0.4210,0.4791,0.4365
"GA approx (rho constant, nu = 0.25)",0.5847,0.6527,0.4355,0.6235,0.4354,0.7639,0.7303,0.3885,0.5268,0.5792,0.5303
"GA approx (rho constant, nu = 0.25 2nd order)",0.7191,0.8143,0.5525,0.7897,0.5358,0.9043,0.8887,0.4803,0.6456,0.7013,0.6761
"GA approx (omega as Input, nu = 0)",0.4911,0.5604,0.3449,0.5312,0.3508,0.6890,0.6691,0.3052,0.4220,0.4823,0.4440
"GA approx (simplified, rho constant, nu = 0.25)",0.5530,0.6184,0.4048,0.5895,0.4112,0.7374,0.7247,0.3624,0.4870,0.5457,0.5028
"GA approx (simplified, rho constant, nu = 0)",0.4866,0.5538,0.3425,0.5238,0.3485,0.6826,0.6684,0.3033,0.4210,0.4791,0.4365
GA CR+ NN,0.2180,0.2487,0.1721,0.2432,0.1796,0.2990,0.2688,0.1568,0.1333,0.3063,0.1970
"GA Monte Carlo (CR+, rho from IRB, nu = 0.25)",0.4204,0.4484,0.2891,0.4223,0.3342,0.5614,0.6066,0.2629,0.3347,0.3989,0.3803
"GA Monte Carlo (CR+, omega as Input, nu = 0.25)",0.4150,0.4488,0.2802,0.4207,0.3360,0.5600,0.5998,0.2702,0.3400,0.3896,0.3714


## 2.) q = 99.9%, ELGD = 0.45

In [19]:
df_999 = compute_GA(q=0.999,ELGD_const = 0.45,
               portfolios = portfolios_0999,
               model_mtm = model_mtm_0999,
               model_actuarial = model_actuarial_0999)
display(df_999)

1/1 [==============================] - 0s 22ms/step


C:\Users\jul_ses\OneDrive - National University of Singapore\Projects\MDB Empirical\Code\misc_functions.py:130: RuntimeWarning: invalid value encountered in sqrt
  K = ((ELGD)*norm.cdf(np.sqrt(1/(1-R_corr))*norm.ppf(PD)+np.sqrt(R_corr/(1-R_corr))*norm.ppf(q))-PD*ELGD)*(1/(1-1.5*b))*(1+(M-2.5)*b)
C:\Users\jul_ses\Anaconda3\envs\tf_cpu\lib\site-packages\scipy\optimize\_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


1/1 [==============================] - 0s 21ms/step


,CAF,ADB,AFDB,IDB,CDB,CABEI,EADB,IBRD,TDB,BOAD,EBRD
Borrowers,16.0000,38.0000,29.0000,26.0000,16.0000,11.0000,4.0000,78.0000,21.0000,8.0000,37.0000
"GA approx (rho from IRB, VLGD = 0)",0.1930,0.1284,0.1060,0.1623,0.1511,0.3933,0.3690,0.0469,0.2246,0.2200,0.0994
"GA approx (omega as Input , VLGD = 0)",0.1977,0.1311,0.1053,0.1664,0.1522,0.3979,0.3690,0.0470,0.2252,0.2200,0.0974
"GA approx (rho from IRB, nu = 0.25)",0.2878,0.1932,0.1568,0.2440,0.2188,0.5925,0.4997,0.0679,0.3453,0.3293,0.1449
"GA approx (rho from IRB, nu = 0.25, 2nd order)",0.8491,0.6992,0.3692,0.8880,0.5170,2.9138,3.0020,0.1347,0.8358,0.8333,0.4044
"GA approx (simplified, nu = 0.25)",0.2519,0.1677,0.1384,0.2119,0.1972,0.5135,0.4818,0.0612,0.2933,0.2872,0.1297
"GA approx (simplified, nu = 0)",0.1930,0.1284,0.1060,0.1623,0.1511,0.3933,0.3690,0.0469,0.2246,0.2200,0.0994
GA CR+ NN,0.0488,0.0300,0.0363,0.0360,0.0581,0.0639,0.0481,0.0159,0.0479,0.0730,0.0280
"GA Monte Carlo (CR+, rho from IRB, nu = 0.25)",0.1092,0.0652,0.0706,0.0809,0.1051,0.1702,0.2661,0.0392,0.1123,0.1287,0.0699
"GA Monte Carlo (CR+, omega as Input, nu = 0.25)",0.1073,0.0659,0.0703,0.0806,0.1029,0.1688,0.2665,0.0395,0.1150,0.1337,0.0696


Relative to UL

In [20]:
df_999_UL = compute_GA(q=0.999,ELGD_const = 0.45,
               portfolios = portfolios_0999,
               model_mtm = model_mtm_0999,
               model_actuarial = model_actuarial_0999,
               compute_mtm= False,
               relative_to_UL = True)
display(df_999_UL)

1/1 [==============================] - 0s 29ms/step


C:\Users\jul_ses\AppData\Local\Temp\ipykernel_26548\940344609.py:53: RuntimeWarning: invalid value encountered in sqrt
  K = ((ELGD)*norm.cdf(np.sqrt(1/(1-R_corr))*norm.ppf(PD)+np.sqrt(R_corr/(1-R_corr))*norm.ppf(q))-PD*ELGD)
C:\Users\jul_ses\Anaconda3\envs\tf_cpu\lib\site-packages\scipy\optimize\_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


1/1 [==============================] - 0s 28ms/step


C:\Users\jul_ses\OneDrive - National University of Singapore\Projects\MDB Empirical\Code\misc_functions.py:130: RuntimeWarning: invalid value encountered in sqrt
  K = ((ELGD)*norm.cdf(np.sqrt(1/(1-R_corr))*norm.ppf(PD)+np.sqrt(R_corr/(1-R_corr))*norm.ppf(q))-PD*ELGD)*(1/(1-1.5*b))*(1+(M-2.5)*b)


,CAF,ADB,AFDB,IDB,CDB,CABEI,EADB,IBRD,TDB,BOAD,EBRD
Borrowers,16.0000,38.0000,29.0000,26.0000,16.0000,11.0000,4.0000,78.0000,21.0000,8.0000,37.0000
"GA approx (rho from IRB, VLGD = 0)",0.6978,0.7176,0.5510,0.7055,0.5884,0.8171,0.8204,0.4848,0.6354,0.6733,0.6156
"GA approx (omega as Input , VLGD = 0)",0.7103,0.7357,0.5551,0.7199,0.5927,0.8208,0.8204,0.5047,0.6366,0.6733,0.5918
"GA approx (rho from IRB, nu = 0.25)",0.7749,0.7926,0.6448,0.7827,0.6743,0.8706,0.8609,0.5768,0.7282,0.7551,0.7003
"GA approx (rho from IRB, nu = 0.25, 2nd order)",0.8491,0.6992,0.3692,0.8880,0.5170,2.9138,3.0020,0.1347,0.8358,0.8333,0.4044
"GA approx (simplified, nu = 0.25)",0.7509,0.7683,0.6157,0.7577,0.6511,0.8536,0.8564,0.5513,0.6947,0.7290,0.6764
"GA approx (simplified, nu = 0)",0.6978,0.7176,0.5510,0.7055,0.5884,0.8171,0.8204,0.4848,0.6354,0.6733,0.6156
GA CR+ NN,0.3771,0.3892,0.3006,0.3573,0.3571,0.4240,0.3733,0.2565,0.2713,0.4062,0.2945
"GA Monte Carlo (CR+, rho from IRB, nu = 0.25)",0.5623,0.5650,0.4469,0.5510,0.4852,0.6655,0.7671,0.4356,0.4791,0.5528,0.5221
"GA Monte Carlo (CR+, omega as Input, nu = 0.25)",0.5727,0.5861,0.4598,0.5608,0.5008,0.6607,0.7646,0.4694,0.4787,0.5487,0.5060


### 2.1) Calibrate Xi

In [21]:
maturity_for_missing_vals = np.mean([5.09,8.20,5.62,8.48,5.39,2.82,7.08,1.64,4.59]) # determine a maturity that should be used for the banks without information
maturities = np.array([5.09,8.20,5.62,8.48,maturity_for_missing_vals,5.39,2.82,7.08,1.64,4.59,maturity_for_missing_vals])
calibrated_xi = calibrate_xi(q=0.999,ELGD_const = 0.45,
               portfolios = portfolios_0999,
               model_mtm = model_mtm_0999,
               model_actuarial = model_actuarial_0999,
              maturities = maturities)
print("The calibrated xi is {}".format(calibrated_xi))

The calibrated xi is 0.09724697117663139


Use the calibrated xi (with maturity 1)

In [22]:
df_999_xi = compute_GA(q=0.999,ELGD_const = 0.45,
               portfolios = portfolios_0999,
               model_mtm = model_mtm_0999,
               model_actuarial = model_actuarial_0999,
               xi = calibrated_xi)
display(df_999_xi)

1/1 [==============================] - 0s 47ms/step


C:\Users\jul_ses\OneDrive - National University of Singapore\Projects\MDB Empirical\Code\misc_functions.py:130: RuntimeWarning: invalid value encountered in sqrt
  K = ((ELGD)*norm.cdf(np.sqrt(1/(1-R_corr))*norm.ppf(PD)+np.sqrt(R_corr/(1-R_corr))*norm.ppf(q))-PD*ELGD)*(1/(1-1.5*b))*(1+(M-2.5)*b)
C:\Users\jul_ses\Anaconda3\envs\tf_cpu\lib\site-packages\scipy\optimize\_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


1/1 [==============================] - 0s 27ms/step


C:\Users\jul_ses\OneDrive - National University of Singapore\Projects\MDB Empirical\Code\misc_functions.py:130: RuntimeWarning: invalid value encountered in sqrt
  K = ((ELGD)*norm.cdf(np.sqrt(1/(1-R_corr))*norm.ppf(PD)+np.sqrt(R_corr/(1-R_corr))*norm.ppf(q))-PD*ELGD)*(1/(1-1.5*b))*(1+(M-2.5)*b)
C:\Users\jul_ses\Anaconda3\envs\tf_cpu\lib\site-packages\scipy\optimize\_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


1/1 [==============================] - ETA: 0s

...

C:\Users\jul_ses\OneDrive - National University of Singapore\Projects\MDB Empirical\Code\misc_functions.py:130: RuntimeWarning: invalid value encountered in sqrt
  K = ((ELGD)*norm.cdf(np.sqrt(1/(1-R_corr))*norm.ppf(PD)+np.sqrt(R_corr/(1-R_corr))*norm.ppf(q))-PD*ELGD)*(1/(1-1.5*b))*(1+(M-2.5)*b)
C:\Users\jul_ses\Anaconda3\envs\tf_cpu\lib\site-packages\scipy\optimize\_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


1/1 [==============================] - 0s 19ms/step


C:\Users\jul_ses\OneDrive - National University of Singapore\Projects\MDB Empirical\Code\misc_functions.py:130: RuntimeWarning: invalid value encountered in sqrt
  K = ((ELGD)*norm.cdf(np.sqrt(1/(1-R_corr))*norm.ppf(PD)+np.sqrt(R_corr/(1-R_corr))*norm.ppf(q))-PD*ELGD)*(1/(1-1.5*b))*(1+(M-2.5)*b)
C:\Users\jul_ses\Anaconda3\envs\tf_cpu\lib\site-packages\scipy\optimize\_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


1/1 [==============================] - 0s 27ms/step


,CAF,ADB,AFDB,IDB,CDB,CABEI,EADB,IBRD,TDB,BOAD,EBRD
Borrowers,16.0000,38.0000,29.0000,26.0000,16.0000,11.0000,4.0000,78.0000,21.0000,8.0000,37.0000
"GA approx (rho from IRB, VLGD = 0)",0.1605,0.1070,0.0880,0.1352,0.1251,0.3277,0.3020,0.0388,0.1879,0.1831,0.0823
"GA approx (omega as Input , VLGD = 0)",0.1296,0.0832,0.0691,0.1050,0.1040,0.2556,0.2794,0.0329,0.1488,0.1515,0.0806
"GA approx (rho from IRB, nu = 0.25)",0.2388,0.1606,0.1299,0.2028,0.1807,0.4925,0.4080,0.0561,0.2880,0.2734,0.1199
"GA approx (rho from IRB, nu = 0.25, 2nd order)",0.6307,0.5156,0.2781,0.6552,0.3853,2.1238,2.0786,0.1021,0.6342,0.6254,0.2996
"GA approx (simplified, nu = 0.25)",0.2096,0.1397,0.1149,0.1765,0.1633,0.4278,0.3943,0.0506,0.2453,0.2390,0.1075
"GA approx (simplified, nu = 0)",0.1605,0.1070,0.0880,0.1352,0.1251,0.3277,0.3020,0.0388,0.1879,0.1831,0.0823
GA CR+ NN,0.0488,0.0300,0.0363,0.0360,0.0581,0.0639,0.0481,0.0159,0.0479,0.0730,0.0280
"GA Monte Carlo (CR+, rho from IRB, nu = 0.25)",0.1061,0.0665,0.0716,0.0808,0.1002,0.1718,0.2627,0.0385,0.1129,0.1297,0.0702
"GA Monte Carlo (CR+, omega as Input, nu = 0.25)",0.1081,0.0654,0.0705,0.0796,0.1060,0.1712,0.2666,0.0404,0.1132,0.1300,0.0701


### 2.2) Constant Rho, Rho = 0.12

In [ ]:
df_999_rho12 = compute_GA(q=0.999,ELGD_const = 0.45,
               portfolios = portfolios_0999_rho012,
               model_mtm = model_mtm_0999,
               model_actuarial = model_actuarial_0999,
               constant_rho = True,
               constant_rho_val = 0.12)
display(df_999_rho12)

1/1 [==============================] - 0s 33ms/step


Relative to UL

In [ ]:
df_999_rho12_UL = compute_GA(q=0.999,ELGD_const = 0.45,
               portfolios = portfolios_0999_rho012,
               model_mtm = model_mtm_0999,
               model_actuarial = model_actuarial_0999,
               constant_rho = True,
               constant_rho_val = 0.12,
               compute_mtm= False,
               relative_to_UL = True)
display(df_999_rho12_UL)

### 2.3) Constant Rho, Rho = 0.24

In [ ]:
df_999_rho24 = compute_GA(q=0.999,ELGD_const = 0.45,
               portfolios = portfolios_0999_rho024,
               model_mtm = model_mtm_0999,
               model_actuarial = model_actuarial_0999,
               constant_rho = True,
               constant_rho_val = 0.24)
display(df_999_rho24)

Relative to UL

In [ ]:
df_999_rho24_UL = compute_GA(q=0.999,ELGD_const = 0.45,
               portfolios = portfolios_0999_rho024,
               model_mtm = model_mtm_0999,
               model_actuarial = model_actuarial_0999,
               constant_rho = True,
               constant_rho_val = 0.24,
               compute_mtm= False,
               relative_to_UL = True)
display(df_999_rho24_UL)

### 2.4) Constant Rho, Rho = 0.35

In [ ]:
df_999_rho = compute_GA(q=0.999,ELGD_const = 0.45,
               portfolios = portfolios_0999_rho035,
               model_mtm = model_mtm_0999,
               model_actuarial = model_actuarial_0999,
               constant_rho = True,
               constant_rho_val = 0.35)
display(df_999_rho)

Relative to UL

In [ ]:
df_999_rho_UL = compute_GA(q=0.999,ELGD_const = 0.45,
               portfolios = portfolios_0999_rho035,
               model_mtm = model_mtm_0999,
               model_actuarial = model_actuarial_0999,
               constant_rho = True,
               constant_rho_val = 0.35,
               compute_mtm= False,
               relative_to_UL = True)
display(df_999_rho_UL)

### 2.5) Illustrate the dependence on $\rho$ in a figure

In [ ]:
MDBs = bank_names

# Plotting
plt.figure(figsize=(10, 6))
plt.scatter(MDBs, df_999_rho12.loc["GA Monte Carlo (IRB model, rho constant, LGD constant)"][MDBs], label="rho = 12%", marker='o')
plt.scatter(MDBs, df_999_rho24.loc["GA Monte Carlo (IRB model, rho constant, LGD constant)"][MDBs], label="rho = 24%", marker='s')
plt.scatter(MDBs, df_999_rho.loc["GA Monte Carlo (IRB model, rho constant, LGD constant)"][MDBs], label="rho = 35%", marker='^')

# Adding labels and title
plt.xlabel("MDBs (Bank Names)")
plt.ylabel("GA")  # Replace with the actual label
plt.title("Granularity Adjustment for Different Rho Values")

# Adding grid
plt.grid(True, linestyle='--', alpha=0.7)

# Adding legend
plt.legend()

# Saving and showing the plot
plt.tight_layout()

plt.savefig("rho_dependence_all_mdbs.pdf", dpi=300)
plt.show()

In [ ]:
MDBs = bank_names

# Plotting
plt.figure(figsize=(10, 6))
plt.scatter(MDBs, df_999_rho12_UL.loc["GA Monte Carlo (IRB model, rho constant, LGD constant)"][MDBs], label="rho = 12%", marker='o')
plt.scatter(MDBs, df_999_rho24_UL.loc["GA Monte Carlo (IRB model, rho constant, LGD constant)"][MDBs], label="rho = 24%", marker='s')
plt.scatter(MDBs, df_999_rho_UL.loc["GA Monte Carlo (IRB model, rho constant, LGD constant)"][MDBs], label="rho = 35%", marker='^')

# Adding labels and title
plt.xlabel("MDBs (Bank Names)")
plt.ylabel("GA/(GA+UL)")  # Replace with the actual label
plt.title("GA contribution to UL for Different Rho Values")

# Adding grid
plt.grid(True, linestyle='--', alpha=0.7)

# Adding legend
plt.legend()

# Saving and showing the plot
plt.tight_layout()

plt.savefig("rho_dependence_relative_all_mdbs.pdf", dpi=300)
plt.show()

### 2.6 ) Dependence on maturity 

In [ ]:
maturities = np.arange(1,11)
maturity_results = []
for maturity in maturities:
    maturity_results.append(compute_GA(q=0.999,ELGD_const = 0.45,
               portfolios = portfolios_0999,
               model_mtm = model_mtm_0999,
               model_actuarial = model_actuarial_0999,
                                      maturity = maturity))

Plot the results

In [ ]:
MDBs = bank_names

# Plotting
plt.figure(figsize=(10, 6))
for bank in MDBs:
    plt.plot(maturities, [maturity_results[m].loc["GA MTM Monte Carlo, nu = 0.25"][bank] for m in range(len(maturities))], label=bank)

# Adding labels and title
plt.xlabel("Maturity")
plt.ylabel("GA")  # Replace with the actual label
plt.title("GA in dependence of Maturity (MtM Monte Carlo)")

# Adding grid
plt.grid(True, linestyle='--', alpha=0.7)

# Adding legend
plt.legend()

# Saving and showing the plot
plt.tight_layout()

plt.savefig("mdbs_dependence_on_maturity_mtm.pdf", dpi=300)
plt.show()

In [ ]:
MDBs = bank_names

# Plotting
plt.figure(figsize=(10, 6))
for bank in MDBs:
    plt.plot(maturities, [maturity_results[m].loc["GA MTM Approx. , nu = 0.25"][bank] for m in range(len(maturities))], label=bank)

# Adding labels and title
plt.xlabel("Maturity")
plt.ylabel("GA")  # Replace with the actual label
plt.title("GA in dependence of Maturity (MtM approx)")

# Adding grid
plt.grid(True, linestyle='--', alpha=0.7)

# Adding legend
plt.legend()

# Saving and showing the plot
plt.tight_layout()

plt.savefig("mdbs_dependence_on_maturity_mtm_approx.pdf", dpi=300)
plt.show()

In [ ]:
MDBs = bank_names

# Plotting
plt.figure(figsize=(10, 6))
for bank in MDBs:
    plt.plot(maturities, [maturity_results[m].loc["GA approx (rho from IRB, nu = 0.25)"][bank] for m in range(len(maturities))], label=bank)

# Adding labels and title
plt.xlabel("Maturity")
plt.ylabel("GA")  # Replace with the actual label
plt.title("GA in dependence of Maturity (CR+ approx)")

# Adding grid
plt.grid(True, linestyle='--', alpha=0.7)

# Adding legend
plt.legend()

# Saving and showing the plot
plt.tight_layout()

plt.savefig("mdbs_dependence_on_maturity_cr_approx.pdf", dpi=300)
plt.show()

In [ ]:
maturity_results[0]

In [ ]:
MDBs = bank_names

# Plotting
plt.figure(figsize=(10, 6))
for bank in MDBs:
    plt.plot(maturities, [maturity_results[m].iloc[1][bank] for m in range(len(maturities))], label=bank)

# Adding labels and title
plt.xlabel("Maturity")
plt.ylabel("GA")  # Replace with the actual label
plt.title("GA in dependence of Maturity (CR+ approx, simplified)")

# Adding grid
plt.grid(True, linestyle='--', alpha=0.7)

# Adding legend
plt.legend()

# Saving and showing the plot
plt.tight_layout()

plt.savefig("mdbs_dependence_on_maturity_cr_approx_simplified.pdf", dpi=300)
plt.show()

In [ ]:
MDBs = bank_names

# Plotting
plt.figure(figsize=(10, 6))
for bank in MDBs:
    plt.plot(maturities, [maturity_results[m].loc["GA Monte Carlo (IRB, nu = 0.25)"][bank] for m in range(len(maturities))], label=bank)

# Adding labels and title
plt.xlabel("Maturity")
plt.ylabel("GA")  # Replace with the actual label
plt.title("GA in dependence of Maturity (IRB MC)")

# Adding grid
plt.grid(True, linestyle='--', alpha=0.7)

# Adding legend
plt.legend()

# Saving and showing the plot
plt.tight_layout()

plt.savefig("mdbs_dependence_on_maturity_irb.pdf", dpi=300)
plt.show()

## 3.)  q = 99%, ELGD = 0.1

In [ ]:
df_99_lgd01 = compute_GA(q=0.99,ELGD_const = 0.1,
               portfolios = portfolios_099,
               model_mtm = model_mtm_099,
               model_actuarial = model_actuarial_099)
display(df_99_lgd01)

Relative to UL

In [ ]:
df_99_lgd01_UL = compute_GA(q=0.99,ELGD_const = 0.1,
               portfolios = portfolios_099,
               model_mtm = model_mtm_099,
               model_actuarial = model_actuarial_099,
               compute_mtm= False,
               relative_to_UL = True)
display(df_99_lgd01_UL)

### 3.1) Constant Rho, Rho = 0.35

In [ ]:
df_99_lgd01_rho = compute_GA(q=0.99,ELGD_const = 0.45,
               portfolios = portfolios_099_rho035,
               model_mtm = model_mtm_099,
               model_actuarial = model_actuarial_099,
               constant_rho = True,
               constant_rho_val = 0.35)
display(df_99_lgd01_rho)

Relative to UL

In [ ]:
df_99_lgd01_rho_UL = compute_GA(q=0.99,ELGD_const = 0.45,
               portfolios = portfolios_099_rho035,
               model_mtm = model_mtm_099,
               model_actuarial = model_actuarial_099,
               constant_rho = True,
               constant_rho_val = 0.35,
               compute_mtm= False,
               relative_to_UL = True)
display(df_99_lgd01_rho_UL)

## 4.) q = 99.9 %, ELGD = 0.1

In [ ]:
df_999_lgd01 = compute_GA(q=0.999,ELGD_const = 0.1,
               portfolios = portfolios_0999,
               model_mtm = model_mtm_0999,
               model_actuarial = model_actuarial_0999,
                          trans_dict = RC_dict)
display(df_999_lgd01)

Relative to UL

In [ ]:
df_999_lgd01_UL = compute_GA(q=0.999,ELGD_const = 0.1,
               portfolios = portfolios_0999,
               model_mtm = model_mtm_0999,
               model_actuarial = model_actuarial_0999,
                compute_mtm= False,
               relative_to_UL = True,
                          trans_dict = RC_dict)
display(df_999_lgd01_UL )            

Calibration of Xi

In [ ]:
maturity_for_missing_vals = np.mean([5.09,8.20,5.62,8.48,5.39,2.82,7.08,1.64,4.59]) # determine a maturity that should be used for the banks without information
maturities = np.array([5.09,8.20,5.62,8.48,maturity_for_missing_vals,5.39,2.82,7.08,1.64,4.59,maturity_for_missing_vals])
calibrated_xi = calibrate_xi(q=0.999,ELGD_const = 0.1,
               portfolios = portfolios_0999,
               model_mtm = model_mtm_0999,
               model_actuarial = model_actuarial_0999,
              maturities = maturities)
print("The calibrated xi is {}".format(calibrated_xi))

### 4.1) Constant Rho, Rho = 0.35

In [ ]:
df_999_lgd01_rho = compute_GA(q=0.999,ELGD_const = 0.1,
               portfolios = portfolios_0999_rho035,
               model_mtm = model_mtm_0999,
               model_actuarial = model_actuarial_0999,
               constant_rho = True,
               constant_rho_val = 0.35,
                          trans_dict = RC_dict)
display(df_999_lgd01_rho)

Relative to UL

In [ ]:
df_999_lgd01_rho_UL = compute_GA(q=0.999,ELGD_const = 0.1,
               portfolios = portfolios_0999_rho035,
               model_mtm = model_mtm_0999,
               model_actuarial = model_actuarial_0999,
               constant_rho = True,
               constant_rho_val = 0.35,
                compute_mtm= False,
               relative_to_UL = True,
                          trans_dict = RC_dict)
display(df_999_lgd01_rho_UL)

# GA Computations for RC Transition Matrix

##  5) q = 99%, ELGD = 0.45

In [ ]:
df_99_RC = compute_GA(q=0.99,ELGD_const = 0.45,
               portfolios = portfolios_099_RC,
               model_mtm = model_mtm_099,
               model_actuarial = model_actuarial_099,compute_mtm=False,
                          trans_dict = RC_dict)
display(df_99_RC)

Relative to UL

In [ ]:
df_99_RC_UL = compute_GA(q=0.99,ELGD_const = 0.45,
               portfolios = portfolios_099_RC,
               model_mtm = model_mtm_099,
               model_actuarial = model_actuarial_099,
                compute_mtm= False,
               relative_to_UL = True,
                          trans_dict = RC_dict)
display(df_99_RC_UL)

### 5.1) Constant Rho, Rho = 0.35

In [ ]:
df_99_RC_rho = compute_GA(q=0.99,ELGD_const = 0.45,
               portfolios = portfolios_099_RC_rho035,
               model_mtm = model_mtm_099,
               model_actuarial = model_actuarial_099,
               constant_rho = True,
               constant_rho_val = 0.35,compute_mtm=True,
                          trans_dict = RC_dict)
display(df_99_RC_rho)

Relative to UL

In [ ]:
df_99_RC_rho_UL = compute_GA(q=0.99,ELGD_const = 0.45,
               portfolios = portfolios_099_RC_rho035,
               model_mtm = model_mtm_099,
               model_actuarial = model_actuarial_099,
               constant_rho = True,
               constant_rho_val = 0.35,compute_mtm=False,
               relative_to_UL = True,
                          trans_dict = RC_dict)
display(df_99_RC_rho_UL)

## 6.) q = 99.9%, ELGD = 0.45

In [ ]:
df_999_RC = compute_GA(q=0.999,ELGD_const = 0.45,
               portfolios = portfolios_0999_RC,
               model_mtm = model_mtm_0999,
               model_actuarial = model_actuarial_0999,compute_mtm=True,
                          trans_dict = RC_dict)
display(df_999_RC)

Relative to UL

In [ ]:
df_999_RC_UL = compute_GA(q=0.999,ELGD_const = 0.45,
               portfolios = portfolios_0999_RC,
               model_mtm = model_mtm_0999,
               model_actuarial = model_actuarial_0999,compute_mtm=False,
               relative_to_UL = True,
                          trans_dict = RC_dict)
display(df_999_RC_UL)

### 6.1) Constant Rho, Rho = 0.35

In [ ]:
df_999_RC_rho = compute_GA(q=0.999,ELGD_const = 0.45,
               portfolios = portfolios_0999_RC_rho035,
               model_mtm = model_mtm_0999,
               model_actuarial = model_actuarial_0999,
               constant_rho = True,
               constant_rho_val = 0.35,compute_mtm=True,
                          trans_dict = RC_dict)
display(df_999_RC_rho)

Relative to UL

In [ ]:
df_999_RC_rho_UL = compute_GA(q=0.999,ELGD_const = 0.45,
               portfolios = portfolios_0999_RC_rho035,
               model_mtm = model_mtm_0999,
               model_actuarial = model_actuarial_0999,
               constant_rho = True,
               constant_rho_val = 0.35,compute_mtm=False,
               relative_to_UL = True,
                          trans_dict = RC_dict)
display(df_999_RC_rho_UL)

## 7) q = 99%, ELGD = 0.1

In [ ]:
df_99_lgd01_RC = compute_GA(q=0.99,ELGD_const = 0.1,
               portfolios = portfolios_099_RC,
               model_mtm = model_mtm_099,
               model_actuarial = model_actuarial_099,compute_mtm=True,
                          trans_dict = RC_dict)
display(df_99_lgd01_RC)

Relative to UL

In [ ]:
df_99_lgd01_RC_UL = compute_GA(q=0.99,ELGD_const = 0.1,
               portfolios = portfolios_099_RC,
               model_mtm = model_mtm_099,
               model_actuarial = model_actuarial_099,compute_mtm=False,
               relative_to_UL = True,
                          trans_dict = RC_dict)
display(df_99_lgd01_RC_UL)

### 7.1) Constant Rho, Rho = 0.35

In [ ]:
df_99_lgd01_RC_rho = compute_GA(q=0.99,ELGD_const = 0.45,
               portfolios = portfolios_099_RC_rho035,
               model_mtm = model_mtm_099,
               model_actuarial = model_actuarial_099,
               constant_rho = True,
               constant_rho_val = 0.35,compute_mtm=True,
                          trans_dict = RC_dict)
display(df_99_lgd01_RC_rho)

Relative to UL

In [ ]:
df_99_lgd01_RC_rho_UL = compute_GA(q=0.99,ELGD_const = 0.45,
               portfolios = portfolios_099_RC_rho035,
               model_mtm = model_mtm_099,
               model_actuarial = model_actuarial_099,
               constant_rho = True,
               constant_rho_val = 0.35,compute_mtm=False,
               relative_to_UL = True,
                          trans_dict = RC_dict)
display(df_99_lgd01_RC_rho_UL )

## 8.) q = 99.9 %, ELGD = 0.1

In [ ]:
df_999_lgd01_RC = compute_GA(q=0.999,ELGD_const = 0.1,
               portfolios = portfolios_0999_RC,
               model_mtm = model_mtm_0999,
               model_actuarial = model_actuarial_0999,compute_mtm=True,
                          trans_dict = RC_dict)
display(df_999_lgd01_RC)

Relative to UL

In [ ]:
df_999_lgd01_RC_UL = compute_GA(q=0.999,ELGD_const = 0.1,
               portfolios = portfolios_0999_RC,
               model_mtm = model_mtm_0999,
               model_actuarial = model_actuarial_0999,compute_mtm=False,
               relative_to_UL = True,
                          trans_dict = RC_dict)
display(df_999_lgd01_RC_UL)

###  8.1) Constant Rho, Rho = 0.35

In [ ]:
df_999_lgd01_RC_rho = compute_GA(q=0.999,ELGD_const = 0.1,
               portfolios = portfolios_0999_RC_rho035,
               model_mtm = model_mtm_0999,
               model_actuarial = model_actuarial_0999,
               constant_rho = True,
               constant_rho_val = 0.35,compute_mtm=True,
                          trans_dict = RC_dict)
display(df_999_lgd01_RC_rho)

Relative to UL

In [ ]:
df_999_lgd01_RC_rho_UL = compute_GA(q=0.999,ELGD_const = 0.1,
               portfolios = portfolios_0999_RC_rho035,
               model_mtm = model_mtm_0999,
               model_actuarial = model_actuarial_0999,
               constant_rho = True,
               constant_rho_val = 0.35,compute_mtm=False,
               relative_to_UL = True,
                          trans_dict = RC_dict)
display(df_999_lgd01_RC_rho_UL)